In [11]:
import random
import pandas as pd
from tqdm import tqdm

In [12]:
Qtable = []
cells=12
actions = ['left', 'down', 'right', 'up']
for state in range(cells):
    action = [0] * len(actions)  
    Qtable.append(action)
Qtable=pd.DataFrame(Qtable)

In [13]:
Qtable

,0,1,2,3
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


In [14]:
currentpos=[0,1]
targetpos=[3,2]
fire=[3,1]
block=[1,1]

In [26]:
import random
import pandas as pd
from tqdm import tqdm

# Grid size
GRID_ROWS, GRID_COLS = 3, 4  # 3 rows, 4 columns

# Define positions
currentpos = (0, 1)   # Start position
targetpos = (2, 2)    # Goal position
fire = [(1, 3)]       # Fire (bad zone)
block = [(1, 1)]      # Obstacles (green block)

# Actions
actions = ["up", "down", "left", "right"]
action_map = {
    "up": (-1, 0),
    "down": (1, 0),
    "left": (0, -1),
    "right": (0, 1)
}

# Map states to numeric indices (0-11)
state_to_index = {(r, c): r * GRID_COLS + c for r in range(GRID_ROWS) for c in range(GRID_COLS)}
index_to_state = {v: k for k, v in state_to_index.items()}

# Initialize Q-table with numeric state indices (0-11) as rows and actions as columns
Qtable = pd.DataFrame(0, index=range(GRID_ROWS * GRID_COLS), columns=actions)

# Hyperparameters
learning_rate = 0.1
discount_factor = 0.99
epsilon = 1.0  
epsilon_limit = 0.01
epsilon_decay = 0.995
episodes = 1000

# Reward function
def get_reward(state):
    if state == targetpos:
        return 500  # Large reward for reaching the target
    elif state in fire:
        return -500  # Large penalty for stepping into the fire (end scenario)
    elif state in block:
        return -100  # Penalty for stepping into a block (stays in place, but still penalized)
    else:
        return -5  # Small penalty for each step

# Environment step function (only allows valid moves)
def step(state, action):
    row, col = state
    move = action_map[action]
    new_row, new_col = row + move[0], col + move[1]

    # Check if the new state is out of bounds or blocked
    if new_row < 0 or new_row >= GRID_ROWS or new_col < 0 or new_col >= GRID_COLS or (new_row, new_col) in block:
        # Stay in the current state if the move is invalid (out of bounds or into block)
        new_state = state
        reward = -10  # Penalize invalid moves
    else:
        new_state = (new_row, new_col)
        reward = get_reward(new_state)

    # Episode ends if the agent reaches the target or steps into the fire
    done = new_state == targetpos or new_state in fire

    return new_state, reward, done

# Q-learning training loop
for episode in tqdm(range(episodes), desc="Training", unit="episode"):
    state = currentpos  # Reset to start position
    done = False

    while not done:
        state_index = state_to_index[state]  # Get the numeric state index (0-11)

        # Choose action (epsilon-greedy)
        if random.uniform(0, 1) < epsilon:
            # Explore: Randomly select only valid actions
            valid_actions = [action for action in actions if step(state, action)[0] != state]
            action = random.choice(valid_actions) if valid_actions else "up"  # If all actions are invalid (stay put)
        else:
            # Exploit: Choose best valid action
            valid_actions = [action for action in actions if step(state, action)[0] != state]
            if valid_actions:
                action = Qtable.loc[state_index, valid_actions].idxmax()  # Choose best valid action
            else:
                action = "up"  # Default to up if no valid actions are available

        # Take step in environment
        new_state, reward, done = step(state, action)

        # Q-learning update
        new_state_index = state_to_index[new_state]  # Get the index of the new state
        max_future_q = Qtable.loc[new_state_index].max()  # Best Q-value for next state
        current_q = Qtable.loc[state_index, action]
        Qtable.loc[state_index, action] += learning_rate * (reward + discount_factor * max_future_q - current_q)

        # Move to next state
        state = new_state

    # Decay epsilon
    epsilon = max(epsilon_limit, epsilon * epsilon_decay)

print("Training complete!")

# Display learned Q-table with state indices (0-11)
print("\nFinal Q-Table (States numbered 0 to 11 and Actions as Columns):")
print(Qtable)


Training:   0%|          | 0/1000 [00:00<?, ?episode/s]C:\Users\georg\AppData\Local\Temp\ipykernel_52248\4060719382.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Qtable.loc[state_index, action] += learning_rate * (reward + discount_factor * max_future_q - current_q)
C:\Users\georg\AppData\Local\Temp\ipykernel_52248\4060719382.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-0.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  Qtable.loc[state_index, action] += learning_rate * (reward + discount_factor * max_future_q - current_q)
C:\Users\georg\AppData\Local\Temp\ipykernel_52248\4060719382.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise a

Training complete!

Final Q-Table (States numbered 0 to 11 and Actions as Columns):
            up        down        left       right
0     0.000000  252.114472    0.000000  470.256132
1     0.000000    0.000000  460.247456  480.100000
2     0.000000  490.000000  469.540765  459.413636
3     0.000000 -470.925131  477.920317    0.000000
4   211.032160  324.799703    0.000000    0.000000
5     0.000000    0.000000    0.000000    0.000000
6   477.729685  500.000000    0.000000 -499.613223
7     0.000000    0.000000    0.000000    0.000000
8   150.133232    0.000000    0.000000  402.685978
9     0.000000    0.000000  109.681479  473.832618
10    0.000000    0.000000    0.000000    0.000000
11    0.000000    0.000000    0.000000    0.000000


In [27]:
Qtable = pd.DataFrame(Qtable)
Qtable.head(13)

,up,down,left,right
0,0.000000,252.114472,0.000000,470.256132
1,0.000000,0.000000,460.247456,480.100000
2,0.000000,490.000000,469.540765,459.413636
3,0.000000,-470.925131,477.920317,0.000000
4,211.032160,324.799703,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000
6,477.729685,500.000000,0.000000,-499.613223
7,0.000000,0.000000,0.000000,0.000000
8,150.133232,0.000000,0.000000,402.685978
9,0.000000,0.000000,109.681479,473.832618
